<a href="https://colab.research.google.com/github/DiegoEspinozaoss/Privacidad_de_Datos_CC5215-1/blob/main/Laboratorio_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CC5215: Privacidad de Datos
## Laboratorio 1

In [ ]:
# Load the data and libraries
import pandas as pd
import numpy as np

adult = pd.read_csv('https://users.dcc.uchile.cl/~mtoro/cursos/cc5215/adult_with_pii.csv')

# Dataset with private information
adult_pii = adult[['Name', 'DOB', 'SSN', 'Zip', 'Age']]
# Deanonimized dataset
adult_deid = adult.drop(columns=['Name', 'SSN'])

In [ ]:
adult.head()

,Name,DOB,SSN,Zip,Age,Workclass,fnlwgt,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,Karrie Trusslove,9/7/1967,732-14-6110,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Brandise Tripony,6/7/1988,150-19-2766,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,Brenn McNeely,8/6/1991,725-59-9860,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Dorry Poter,4/6/2009,659-57-4974,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,Dick Honnan,9/16/1951,220-93-3811,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Question 1 (5 points)

Using the dataframes `adult_pii` and `adult_deid`, write code to conduct a linking attack to recover the names of as many individuals in `adult_deid` as possible. Your solution should be parameterized by the set of columns to use in the attack.

In [ ]:
def linking_attack(cols):
    f_merged = pd.merge(adult_pii, adult_deid, on=cols, how='inner')
    return f_merged
    raise NotImplementedError()

In [ ]:
# TEST CASES for Question 1

assert len(linking_attack(['Zip'])) == 43191
assert len(linking_attack(['Zip', 'DOB'])) == 32563
assert len(linking_attack(['Zip', 'Age'])) == 32755

## Question 2 (5 points)

How many individuals in this dataset are uniquely identified by their Zip code? How many are uniquely identified by their age?

Hint: note that the number of *unique ZIP codes* is **different** from the number of *individuals uniquely identified by ZIP code*.

Hint: you can use the `value_counts` method (and its `subset` parameter) to count the number of occurences of each value in a series.

In [ ]:
def unique_zipcode():
    zip_counts = adult_pii['Zip'].value_counts()
    # Contar ZIPs que aparecen una sola vez
    cantidad_zip_unicos = (zip_counts == 1).sum()
    return cantidad_zip_unicos
    raise NotImplementedError()

def unique_dob():
    DOB_counts = adult_pii['DOB'].value_counts()
    # Contar DOBs que aparecen una sola vez
    cantidad_DOB_unicos = (DOB_counts == 1).sum()
    return cantidad_DOB_unicos
    raise NotImplementedError()

In [ ]:
# TEST CASES for Question 2

assert unique_zipcode() == 23513
assert unique_dob() == 7845

## Question 3 (10 points)

Write code to determine the `Education-Num` of the individual named Ardyce Golby by performing a differencing attack. Your code should *only* use aggregate data to find Ardyce's education number.

In [ ]:
def ardyce_education():
    df_merged = pd.merge(adult_pii, adult_deid, on=['Age', 'DOB', 'Zip'], how='inner')
    find = df_merged[df_merged["Name"]=="Ardyce Golby"]["Education-Num"].sum()
    return find
    raise NotImplementedError()


In [ ]:
# TEST CASE for Question 3
assert ardyce_education() == 12

## Question 4 (15 points)

Implement a more efficient version of `is_k_anonymous`. The inefficient implementation, taken from the textbook, appears below.

**Hint**: use the [`value_counts`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html) or `group_by` functions, and make sure no count is less than $k$.

In [ ]:
# Checking for k-Anonymity, taken from the textbook
#def is_k_anonymous(k, qis, df):
#     for index, row in df.iterrows():
#         query = ' & '.join([f'`{col}` == "{row[col]}"' for col in qis])
#         rows = df.query(query)
#         if (rows.shape[0] < k):
#             return False
#     return True

In [ ]:
def is_k_anonymous(k, qis, df):
  qis_counts = df.groupby(qis).size()
  min_unicos = qis_counts.min()
  if min_unicos >= k:
    return True
  return False

In [ ]:
# TEST CASES for question 4

assert not is_k_anonymous(2, ['Age'], adult)
assert is_k_anonymous(1, ['Age'], adult)
assert is_k_anonymous(1, ['Age', 'Occupation'], adult)

## Question 5 (10 points)

Implement a `generalize` function that takes a dataframe `df` and a dictionary `depths` that describes how much to generalize each column of `df`. Generalizing a column to a depth of $n$ replaces the $n$ least-significant digits of each number in that column by zeroes. For example, we could generalize column `A` by making its least-significant digit a 0 and column `B` by doing the same for 2 digits with the following depth specification:

In [ ]:
depths = {
    'A': 2,
    'B': 1
}

In [ ]:
def generalize(df, depths):
  # Recorrer el diccionario y truncar los valores en el DataFrame
  for col, decimals in depths.items():
    if col in df.columns:
        factor = 10 ** decimals  # Calcular el factor de truncado
        df[col] = (df[col] // factor) * factor  # Truncar y poner ceros en los dígitos posteriores
  return df
  raise NotImplementedError()

The result of generalizing the age by depth of 1 should satisfy k-anonimity for $k = 20$:

In [ ]:
# Test case for question 5

def generalize_adult_age():
    depths = {
        'Age': 1,
    }
    return generalize(adult[['Age']], depths)

assert is_k_anonymous(20, ['Age'], generalize_adult_age())

<ipython-input-16-9a355410d467>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = (df[col] // factor) * factor  # Truncar y poner ceros en los dígitos posteriores


## Question 6 (5 points)

Using the `generalize` function, generalize the `Age` and `Zip` columns of the `adult` dataset in order to achieve $k$-Anonymity for $k=5$. Your result should drop other columns besides these two.

In [ ]:
def generalize_adult_age_zip():
    depths = {
        'Age': 2,
        'Zip': 2
    }

    return generalize(adult[['Age', 'Zip']], depths)

assert is_k_anonymous(5, ['Age', 'Zip'], generalize_adult_age_zip())

<ipython-input-16-9a355410d467>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = (df[col] // factor) * factor  # Truncar y poner ceros en los dígitos posteriores
<ipython-input-16-9a355410d467>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = (df[col] // factor) * factor  # Truncar y poner ceros en los dígitos posteriores


## Question 7 (10 points)

In 1-4 sentences each, answer the following:

1. How much generalization was required to achieve $k=5$ in question 6?

2. Does this level of generalization significantly impact the utility of the $k$-Anonymized data? Why or why not?

3. Is there another approach, in addition to our simple generalization method, that might work better?
  
4. What is a simple method for generalizing the `Occupation` column?

Your answer here:
1. Para lograr tener 5-anonymous tuvimos que perder gran parte de la información, haciendo que por lo menos una de las columnas todos sus datos fueran 0 y la otra que perdiera por lo menos 2 digitos de 5.
2. Al final la columna de edad queda inutilizable, ya que no tiene información. Y con datos tan generales es dificil poder sacar conclusiones que tengan valor.
3. Aparte de truncar también es posible clasificar otras columnas de datos tipo string en diferentes categorias,  como por ejemplo Occupation puede estar en categorias de que tipo de trabajo o Country puede estar separado por continente.
4. Occupation puede generalizarse creando categorias más generales de trabajo, como Privado, Servicio Publico, FFAA, Estudiante, etc.